## Usando arquivo .env para controlar variaveis de ambiente
Para evitar exposição da chave `OPENAI_API_KEY` optei por utilizar arquivo `.env` com a informação da chave.

Para seguir o mesmo método basta criar um arquivo `.env` no mesmo diretório do arquivo `my-first-crew.ipynb`.
A importação da chave será feita através da célula abaixo que faz a instalação de um biblioteca para carregar
os valores do arquivo `.env`.

### Introdução ao CrewAI

Nesta aula, vamos explorar o CrewAI utilizando um Jupyter Notebook. Começamos importando as principais bibliotecas necessárias, como agent, task e crew. O foco é criar um agente que, com a ajuda de três componentes — um planejador, um escritor e um editor —, será capaz de gerar artigos. Também abordamos a configuração da chave de API da OpenAI, essencial para o funcionamento do nosso projeto. É uma introdução prática ao uso de múltiplos agentes em tarefas de escrita.

In [ ]:
from crewai import Agent, Task, Crew
from IPython.display import Markdown
import dotenv

import os

In [ ]:
os.environ['OPENAI_API_KEY'] = ""
os.environ['OPENAI_MODEL_NAME'] = "gpt-3.5-turbo"

### Criando agentes

Nesta aula, introduzimos a criação de agentes usando a CrewAI. Começamos com o "Planner", que tem um papel de planejador de conteúdo, com o objetivo de gerar conteúdo engajador sobre um tópico específico. Em seguida, discutimos o "Writer", que escreve opiniões baseadas no trabalho do Planner. Por fim, apresentamos o "Editor", responsável por alinhar o texto ao estilo da organização. Cada agente possui um papel, objetivo e backstory definidos, além de parâmetros como "allowDelegation" e "verbose".

**Grupo de agente para escrever artigos**

1. Planner - Planejador do artigo
2. Writer - Escrito do artigo
3. Editor - Editor do artigo

In [21]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging content about the {topic}",
    backstory="""You're working on planning a blog post article about
    the topic: {topic}.
    You collect information that helps the audience learn something
    and mak informed decisions.
    Your work is the basics for the Content Writer to write article on this topic.
    """,
    allow_delegation=False,
    verbose=True,
)

Você está criando um objeto `Agent` chamado `writer` utilizando a dependência `crewai`.

**Classe Agent**: A classe `Agent` não é uma classe built-in do Python, então é provável que seja uma classe personalizada definida em outro lugar no seu código. A classe `Agent` epresenta uma entidade que realiza tarefas ou ter certas características.

**Argumentos de palavra-chave**: O objeto `Agent` é criado com vários argumentos de palavra-chave:

* `role`: uma string que indica o papel do agente, que é definido como `"Content Writer"`.
* `goal`: uma string que descreve o objetivo do agente, que é uma string de template que inclui um placeholder `{topic}`.
* `backstory`: uma string multilinhas que fornece contexto sobre o histórico e comportamento do agente. Essa string também inclui um placeholder `{topic}`.
* `allow_delegation`: um flag booleano definido como `False`, que controla se o agente pode delegar tarefas para outros.
* `verbose`: um flag booleano definido como `True`, que controla o nível de saída ou logging do agente.

**Placeholders**: As strings `goal` e `backstory` contêm placeholders `{topic}`, o que sugere que esses valores serão substituídos por nomes de tópicos reais quando o agente for utilizado.

In [22]:
writer = Agent(
    role="Content Writer",
    goal="Write insightfull and factually accurate opinion piece about the topic: {topic}",
    backstory="""You're working on writing and opinion about the topic: {topic}.
    You base your opinion on the work of the Conntent Planner, who provides an outline and relevant context
    about the topic. You follow the main objectives and direction of the outline, as provided by the Content Planner.
    You also provide objective and impartial insights and back them with information provided the Content Planner.
    """,
    allow_delegation=False,
    verbose=True,
)

 O código define um agente `writer` utilizando a biblioteca `crewai`. Esse agente tem o papel de um **redator de conteúdo** com o objetivo de escrever um artigo opinativo preciso e informativo sobre um determinado **tópico**.  

- **`role`**: Define o papel do agente como "Content Writer".  
- **`goal`**: Especifica o objetivo do agente, que é escrever um artigo opinativo baseado no tópico fornecido (`{topic}`).  
- **`backstory`**: Descreve o contexto do agente, informando que ele baseia sua opinião no trabalho do "Content Planner", que fornece um esboço e informações relevantes. O agente deve seguir as diretrizes estabelecidas pelo "Content Planner" e fornecer insights imparciais com embasamento factual.  
- **`allow_delegation=False`**: Impede que o agente delegue tarefas para outros agentes.  
- **`verbose=True`**: Habilita a exibição detalhada da execução do agente.  

Esse agente é útil em fluxos de trabalho automatizados para geração de conteúdo estruturado e fundamentado.

In [23]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post article to align with writing style of the organization.",
    backstory="""You're are a editor that receives a blog post from the Content Writer.
    Your goal is to review the blog post to ensure that it follow a journalistic best practices.
    """,
    allow_delegation=False,
    verbose=True,
)

O código define um agente `editor` utilizando a biblioteca `crewai`. Esse agente tem o papel de um **editor** responsável por revisar e editar um artigo de blog para garantir que ele esteja alinhado com o estilo da organização.  

- **`role`**: Define o papel do agente como "Editor".  
- **`goal`**: Especifica que o objetivo do agente é editar um artigo de blog para que ele siga o estilo de escrita da organização.  
- **`backstory`**: Descreve que o agente recebe um artigo do "Content Writer" e deve revisá-lo garantindo que siga as melhores práticas jornalísticas.  
- **`allow_delegation=False`**: Impede que o agente delegue tarefas para outros agentes.  
- **`verbose=True`**: Habilita a exibição detalhada da execução do agente.  

Esse agente é útil para garantir a qualidade e padronização do conteúdo antes da publicação.

### Criando tasks

Nesta aula, vamos explorar como criar um plano de conteúdo eficaz para blogs. Começamos identificando o público-alvo, considerando seus interesses e desafios. Em seguida, elaboramos um esboço detalhado que inclui introdução, pontos principais e uma chamada para ação, além de otimização para SEO. Depois, passamos para a redação do post, garantindo que ele tenha uma estrutura envolvente. Por fim, discutimos a importância da edição para corrigir erros e alinhar o texto à voz da marca.

In [24]:
plan = Task(
    description=(
        "1. Prioritaze the last trends, key players, and noteworthy news on {topic}"
        "2. Identify the target audience, considering their interests and pain points."
        "3. Develop a detaild content outline including an introduction, key points, and call to action."
        "4. Include SEO keyword and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document with an outline, audience analysis, SEO keywords and resources",
    agent=planner,
)

O código define uma **tarefa** (`plan`) usando a biblioteca `crewai`. Essa tarefa especifica um conjunto de ações a serem realizadas por um agente chamado `planner`.  

- **`description`**: Lista os passos que a tarefa deve seguir:  
  1. Priorizar tendências, principais atores e notícias relevantes sobre o tópico (`{topic}`).  
  2. Identificar o público-alvo, considerando seus interesses e desafios.  
  3. Criar um esboço detalhado do conteúdo, incluindo introdução, pontos principais e chamada para ação.  
  4. Incluir palavras-chave de SEO e dados ou fontes relevantes.  

- **`expected_output`**: Define a saída esperada da tarefa, que é um **documento de planejamento de conteúdo** contendo:  
  - Estrutura do conteúdo (outline).  
  - Análise do público-alvo.  
  - Palavras-chave de SEO.  
  - Fontes e recursos relevantes.  

- **`agent=planner`**: Especifica que o agente responsável pela execução da tarefa é `planner`.  

Esse código é útil para automatizar a criação de planos de conteúdo bem estruturados e otimizados para SEO.

In [25]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling blog post on topic: {topic}."
        "2. Incorporate the SEO keyword naturally."
        "3. Sections/Subtitles are properly named in engaging manner."
        "4. Ensure the post is scructured with and engaging introduction, insightfull body and summarization conclusion."
    ),
    expected_output="A well-writen blog post in Markdown format, ready for publication, each section should have 2 or 3 paragraphs",
    agent=writer,
)

O código define uma **tarefa** (`write`) usando a biblioteca `crewai`. Essa tarefa instrui o agente `writer` a escrever um **artigo de blog** seguindo um plano de conteúdo.  

- **`description`**: Lista os passos que a tarefa deve seguir:  
  1. Usar o plano de conteúdo para criar um blog post envolvente sobre o tópico (`{topic}`).  
  2. Incorporar palavras-chave de SEO de forma natural.  
  3. Nomear seções e subtítulos de maneira atrativa.  
  4. Estruturar o artigo com:  
     - **Introdução envolvente**  
     - **Corpo informativo e perspicaz**  
     - **Conclusão que resume os principais pontos**  

- **`expected_output`**: Define que a saída esperada é um **artigo bem escrito em formato Markdown**, pronto para publicação. Cada seção deve conter **2 ou 3 parágrafos**.  

- **`agent=writer`**: Define que o agente responsável por essa tarefa é `writer`.  

Esse código automatiza a geração de artigos otimizados para SEO, garantindo uma estrutura coesa e envolvente.

In [26]:
edit = Task(
    description="Proofread the given blog post for grammatical errors and aligment with the brand's voices.",
    expected_output="A well-writen blog post in Markdown format, ready for publication, each section should have 2 or 3 paragraphs",
    agent=editor,
)

O código define uma **tarefa** (`edit`) utilizando a biblioteca `crewai`. Essa tarefa instrui o agente `editor` a revisar um **artigo de blog** para corrigir erros gramaticais e garantir alinhamento com a identidade da marca.  

- **`description`**: Define a tarefa de **revisão** do artigo, com foco em:  
  - Correção de erros gramaticais.  
  - Ajuste do tom e estilo para alinhamento com a voz da marca.  

- **`expected_output`**: Define que a saída esperada é um **artigo revisado em formato Markdown**, pronto para publicação. Cada seção deve conter **2 ou 3 parágrafos**.  

- **`agent=editor`**: Especifica que o agente responsável por essa tarefa é `editor`.  

Esse código garante que o conteúdo final seja **bem escrito, coerente e alinhado à identidade da marca** antes da publicação.

### Criando e utilizando a Crew

Nesta aula, criamos uma "crew" de agentes, que é um grupo de múltiplos agentes, cada um com suas tarefas específicas: Planner, Writer e Editor. Definimos os parâmetros e iniciamos o processo com um tópico, como inteligência artificial. Acompanhamos o debug de cada etapa, mostrando como os agentes colaboram para construir um documento. Utilizamos o CrewAI para gerar um artigo de forma rápida. Nas próximas aulas, vamos explorar como integrar ferramentas externas.

In [29]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2,
)

2025-02-10 22:54:53,475 - 140519159077760 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


O código define uma **equipe** (`crew`) utilizando a biblioteca `crewai`, composta por agentes e tarefas, e configura o nível de verbosidade para a execução.  

- **`agents=[planner, writer, editor]`**: Define os agentes que farão parte da equipe:  
  - `planner`: Responsável por criar o plano de conteúdo.  
  - `writer`: Responsável por escrever o artigo baseado no plano.  
  - `editor`: Responsável por revisar o artigo em termos de gramática e alinhamento com a voz da marca.  

- **`tasks=[plan, write, edit]`**: Define as tarefas que a equipe irá executar:  
  - `plan`: Criação do plano de conteúdo.  
  - `write`: Redação do artigo de blog com base no plano.  
  - `edit`: Revisão do artigo para correção de erros e adequação ao tom da marca.  

- **`verbose=2`**: Configura o nível de verbosidade, o que significa que será exibida uma quantidade detalhada de informações sobre o processo de execução dos agentes e tarefas.  

Esse código organiza e executa um fluxo de trabalho coordenado entre os agentes para criação de conteúdo, desde o planejamento até a revisão final.

In [30]:
result = crew.kickoff(inputs={"topic": "Artificial Inteligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritaze the last trends, key players, and noteworthy news on Artificial Inteligence2. Identify the target audience, considering their interests and pain points.3. Develop a detaild content outline including an introduction, key points, and call to action.4. Include SEO keyword and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan: 
Title: "The Latest Trends in Artificial Intelligence: A Comprehensive Guide"
Outline:
I. Introduction
- Brief overview of the importance of Artificial Intelligence (AI) in today's world
- Introduction to key players and recent advancements in AI technology
II. Key Trends in AI
- Analysis of the latest trends in AI, including machine learning, natural language processing, and robotics
- Discussion on how AI is being used in various industries such as healthcare, finance, and marketing
III. Key Players in AI

O código executa a **equipe** (`crew`) através do método `kickoff` com um **input** específico.  

- **`crew.kickoff(inputs={"topic": "Artificial Intelligence"})`**:  
  - **`kickoff`**: Inicia o processo de execução das tarefas e agentes definidos na equipe.  
  - **`inputs={"topic": "Artificial Intelligence"}`**: Passa o input para a execução das tarefas, onde o tópico do conteúdo é definido como **"Artificial Intelligence"**. Esse valor será usado pelos agentes para gerar o plano de conteúdo, escrever o artigo e revisar o texto.

O comando executa o fluxo completo de planejamento, redação e revisão, com base no tópico fornecido, e o resultado será armazenado em `result`.

In [34]:
Markdown(result)

# The Latest Trends in Artificial Intelligence: A Comprehensive Guide

## Introduction
Artificial Intelligence (AI) has become an integral part of our modern world, revolutionizing industries and shaping the way we interact with technology. From healthcare to finance, AI has made significant advancements in machine learning, natural language processing, and robotics. Key players in the AI industry, such as Google, Amazon, and IBM, have been at the forefront of research and development, pushing the boundaries of what is possible with AI technology.

## Key Trends in AI
The latest trends in AI are driving innovation across various sectors, with machine learning algorithms improving efficiency and accuracy in decision-making processes. Natural language processing has enabled chatbots and virtual assistants to communicate with humans in a more natural and intuitive way. Robotics has also seen significant advancements, with autonomous vehicles and drones revolutionizing transportation and logistics. AI is being used in healthcare to analyze medical images and diagnose diseases, in finance to detect fraud and optimize investments, and in marketing to personalize customer experiences and drive sales.

## Key Players in AI
Leading companies in the AI industry, such as Google, Amazon, and IBM, are investing heavily in AI research and development. Google's DeepMind has made breakthroughs in deep learning and reinforcement learning, while Amazon's Alexa has set the standard for voice-activated assistants. IBM's Watson has been instrumental in advancing AI applications in healthcare and business analytics. These companies continue to push the boundaries of AI technology and drive innovation in the field.

## Noteworthy News in AI
Recent news and breakthroughs in AI technology have captured the attention of the public, with advancements in autonomous vehicles, facial recognition, and natural language processing making headlines. The implications of these advancements on society and the economy are vast, raising questions about ethics, privacy, and job displacement. As AI continues to evolve, it is crucial for policymakers, businesses, and individuals to stay informed and engaged with the latest developments in the field.

## Target Audience Analysis
Professionals in the technology sector, students studying AI, and business owners interested in AI applications form the target audience for this blog post. They are eager to stay updated on the latest trends and news in AI, but may struggle with understanding complex concepts and staying ahead in a rapidly evolving field. By providing valuable insights and analysis, this blog post aims to educate and inform the target audience on the latest advancements in AI technology.

In conclusion, Artificial Intelligence is a rapidly evolving field that is shaping the future of technology and society. By staying informed and engaged with the latest trends, key players, and news in AI, we can better understand the impact of this transformative technology on our lives. Subscribe for more updates on AI trends and news, and share this article with your network to spread awareness about the exciting advancements in Artificial Intelligence.